In [116]:
# loading libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import base64
import cv2
import numpy as np

import warnings
warnings.filterwarnings("ignore")

df=pd.read_pickle("/home/pengtan/Downloads/LSWMD.pkl")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 811457 entries, 0 to 811456
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   waferMap        811457 non-null  object 
 1   dieSize         811457 non-null  float64
 2   lotName         811457 non-null  object 
 3   waferIndex      811457 non-null  float64
 4   trianTestLabel  811457 non-null  object 
 5   failureType     811457 non-null  object 
dtypes: float64(2), object(4)
memory usage: 37.1+ MB


In [117]:
def array2base64str(arr):
    import base64
    b = arr.tobytes()
    e = base64.b64encode(b).decode("UTF-8")
    return e

def base64str2array(s, shape=None):
    import numpy as np
    img_data = base64.b64decode(s)
    nparr = np.fromstring(img_data, np.uint8)
    if shape is None:
        return nparr
    else:
        return nparr.reshape(shape)

def get_failure_type(d):
    try: 
        r = d[0][0]
    except:
        r = 'none'
    return r
    

def get_width(d):
    try:
        s = d.shape
        w = s[0]
    except:
        w = 0
    return w

def get_heigth(d):
    try:
        s = d.shape
        h = s[1]
    except:
        h = 0
    return h

def get_lot_name(n):
    try:
        lot_name = n[0:3] + n[3:].zfill(7)
    except:
        lot_name = "invalid_lot_name"
    return lot_name


## Convert into Base64 String

In [118]:
df['lot_name'] = df['lotName'].apply(get_lot_name)
df['die_size'] = df['dieSize'].astype(int)
df['wafer_index'] = df['waferIndex'].astype(int)
df['failure_type'] = df['failureType'].apply(get_failure_type)
df['wafer_map'] = df['waferMap'].apply(array2base64str)
df['map_width'] = df['waferMap'].apply(get_width)
df['map_heigth'] = df['waferMap'].apply(get_heigth)
df['rowkey'] = df['lot_name'] + "_" + df['wafer_index'].astype(str).str.zfill(2) 

In [119]:
df_data = df[['rowkey', 'lot_name', 'wafer_index', 'failure_type', 'wafer_map', 'map_width', 'map_heigth', 'die_size']]

In [120]:
df_data.head(5)

,rowkey,lot_name,wafer_index,failure_type,wafer_map,map_width,map_heigth,die_size
0,lot0000001_01,lot0000001,1,none,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAgEBAQEBAAAAAAAAAA...,45,48,1683
1,lot0000001_02,lot0000001,2,none,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAQIBAQIBAAAAAAAAAA...,45,48,1683
2,lot0000001_03,lot0000001,3,none,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAQEBAgICAAAAAAAAAA...,45,48,1683
3,lot0000001_04,lot0000001,4,none,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAgECAgICAAAAAAAAAA...,45,48,1683
4,lot0000001_05,lot0000001,5,none,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAQECAQEBAAAAAAAAAA...,45,48,1683


In [123]:
df.shape

(811457, 14)

In [121]:
df_data.to_parquet("wm811k.parquet")